In [1]:
import random
import statistics
import math 
import numpy as np
from numpy import *
import csv
import os
import re
import matplotlib.pyplot as plt
import time

from io import StringIO

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import learning_curve


In [2]:
print (os.getcwd())

/dcs/pg18/u1892296/Downloads


# the path

In [3]:
path = "/dcs/pg18/u1892296/Downloads/stock/"
files = os.listdir(path)
valid_file = []

for file in files:
    if file[-4:] == ".csv":
        valid_file.append(file)
        
print (len(valid_file))

5115


# define the fuctions will be used 

In [4]:
# get the sign of a number
def sgn(x):
    if x > 0:
        return (1)
    else:
        return (0)
    
  
def sgn_0(x):
    if x > 0:
        return (1)
    if x == 0:
        return (0)
    if x < 0:
        return (-1)
    
    
# the number of pos and neg in a list determine the general trend
def sgn_num(x):
    po = 0
    ne = 0
    for i in x:
        if i > 0:
            po += 1
        if i <= 0:
            ne += 1
    if po > ne:
        return (1)
    if po <= ne:
        return (-1)    

# the value of pos and neg sum determine the 
def sgn_total(x):
    po = 0
    ne = 0
    for i in x:
        if i > 0:
            po += i
        if i <= 0:
            ne += abs(i)
    if po > ne:
        return (1)
    if po <= ne:
        return (-1)
    
    
def sqrt_abs(x):
    if x > 0:
        return math.log(x, 10)
    if x == 0:
        return 0
    else:
        return -math.log(abs(x), 10)

In [5]:
def equal_split(l, m):
    n = int(math.ceil(len(l)/float(m)))
    return [l[i:i+n] for i in range(0, len(l), n)]

def chunks(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]

def mix(l):
    res_sum = []
    for i in range(0, len(l[0])):
        res = []
        for lst in l:
            res.append(lst[i])
        res_sum.append(res)
    return res_sum

def cut(l, n):
    res = []
    for i in range(0, n):
        res.append(l[i::n])
    return res

def slic(l, n, m):
    res = []
    for i in range (1, n+1):
        res.append(l[60*i+1:60*i+m+1])
    return res

# building the training model

In [6]:
start = time.clock()

training_data_total = []
training_label_total = []

for file in valid_file:
    
    open_price = []
    high_price = []
    low_price = []
    close_price = []
    volume = []
    change = []
    typical_price = []
    middle_price = []
    
    
    
    
    file_name = path + file
    
    with open(file_name, "rU", encoding = "ISO-8859-1") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:            
            
            diff = float(row[4]) - float(row[1])
            tp = (float(row[2]) + float(row[3]) + float(row[4]))/3
            mp = (float(row[2]) + float(row[3]))/2
        
            # tp = typical price, mp = middle price
            
            row.append(float('%.3f' % diff))
            row.append(float('%.3f' % tp))
            row.append(float('%.3f' % mp))
        
        
            open_price.append(float(row[1]))
            open_price_part = open_price[-700:]
            open_price_list = open_price[-700:-100]
            open_price_list_split = chunks(open_price_list, 60)
            open_price_list_testing = open_price[-100:]
        
        
            high_price.append(float(row[2]))
            high_price_part = high_price[-700:]
            high_price_list = high_price[-700:-100]
            high_price_list_split = chunks(high_price_list, 60)
            high_price_list_testing = high_price[-100:]
            
            low_price.append(float(row[3]))
            low_price_part = low_price[-700:]
            low_price_list = low_price[-700:-100]
            low_price_list_split = chunks(low_price_list, 60)
            low_price_list_testing = low_price[-100:]
            
            close_price.append(float(row[4]))
            close_price_part = close_price[-700:]
            close_price_list = close_price[-700:-100]
            close_price_list_split = chunks(close_price_list, 60)
            close_price_list_testing = close_price[-100:]
            
            volume.append(float(row[5]))
            volume_part = volume[-700:]
            volume_list = volume[-700:-100]
            volume_list_split = chunks(volume_list, 60)
            volume_list_testing = volume[-100:]
            
            change.append(row[7])
            change_part = change[-700:]
            change_list = change[-700:-100]
            change_list_split = chunks(change_list, 60)
            change_list_testing = change[-100:]
            
            typical_price.append(row[8])
            typical_price_part = typical_price[-700:]
            typical_price_list = typical_price[-700:-100]
            typical_price_list_split = chunks(typical_price_list, 60)
            typical_price_list_testing = typical_price[-100:]
            
            middle_price.append(row[9])
            middle_price_part = middle_price[-700:]
            middle_price_list = middle_price[-700:-100]
            middle_price_list_split = chunks(middle_price_list, 60)
            middle_price_list_testing = middle_price[-100:]
    
    training_list = [open_price_list_split, high_price_list_split, low_price_list_split, close_price_list_split, \
                    volume_list_split, change_list_split, typical_price_list_split, middle_price_list_split]

    training_list_mixed = mix(training_list)

    
    
    
    stock_feature_dict = {}
    
    for item in training_list_mixed:
    
        pos_list = []
        abs_list = []
    
        for flt in item[5]:
            abs_list.append(abs(flt))
            if flt > 0:
                pos_list.append(flt)
    
        abs_sum = float('%.3f' % sum(abs_list))
        pos_sum = float('%.3f' % sum(pos_list))
        raw_rsi = (pos_sum/(1+abs_sum))*100
        RSI = float('%.3f' % raw_rsi)
        stock_feature_dict['RSI'] = RSI 
        
        
        
        
        indicator_list = [0.5] 
    
        money_flow_list = [vol*tp for vol, tp in zip(item[4], item[6])]
        total_money_flow = float('%.2f' % sum(money_flow_list))
    
        for i in range(len(item[6])-1):
            det = sgn(float('%.2f' % (item[6][i+1] - item[6][i])))
            indicator_list.append(det)
       
        positive_money_flow_list = [ind*mf for ind, mf in zip(indicator_list, money_flow_list)]
        positive_money_flow = float('%.2f' % sum(positive_money_flow_list))
        negative_money_flow = total_money_flow - positive_money_flow
        money_rate = (positive_money_flow/negative_money_flow)
    
        raw_mfi = 100-100/(1+money_rate)
        MFI = float('%.3f' % raw_mfi)
        stock_feature_dict['MFI'] = MFI   
    
    
        if len(item[3]) >= 3:
            raw_rsv = 100*(close_price_list[-1] - min(close_price_list))/(max(close_price_list) - min(close_price_list))
        RSV = float('%.3f' % raw_rsv)
        stock_feature_dict['RSV'] = RSV
    
    
    
        ax = item[3][-1] - item[3][-5]
        bx = item[3][-5]
        raw_roc = 100*ax/bx
        ROC = float('%.3f' % raw_roc)
        stock_feature_dict['ROC'] = ROC
    
    
        square_sum_5 = []
        TP_5 = mean(item[6][-5:])
        MA_5 = mean(item[3][-5:])
        for i in item[3][-5:]:
            square_sum_5.append((i-MA_5)**2)
        MD_5 = math.sqrt(mean(square_sum_5))
    
        raw_cci = (TP_5 - MA_5)/(MD_5*0.015)
        CCI = float('%.3f' % raw_cci)
        stock_feature_dict["CCI"] = CCI
    
    
        vol_para = list(map(lambda x: (2*x[0]-x[1]-x[2])/exp(x[1]-x[2]), zip(item[3], item[1], item[2])))
    
        va = []
        va_change_list = []
        va.append(item[4][0])
    
        for i in range(0, len(item[4])-1):
            va.append(va[i] + vol_para[i]*item[4][i+1])
    
        for i in range(0, len(va)-1):
            va_change_list.append(va[i+1] - va[i])
        va_change_rate = (va[-1] - va[0])/va[0]
    
        if abs(va_change_rate) > 0.1:
            VA = sgn(va_change_rate)
        else:
            VA = sgn_num(va_change_list)
        stock_feature_dict["VA"] = VA
    
    
    
    
        closing_change_list = []

        for i in range(0, len(item[3])-1):
            closing_change_list.append(item[3][i+1]-item[3][i])
   
        closing_price_list_pvt = item[3][1:]
        volume_list_pvt = volume_list[1:]
        pvt_list = list(map(lambda x: x[0]*x[1]/x[2], zip(closing_change_list, volume_list_pvt, closing_price_list_pvt)))
        raw_pvt = sqrt_abs(sum(pvt_list))
        PVT = float('%.3f' % raw_pvt)
        stock_feature_dict["PVT"] = PVT
    
    
    
    
        sign_closing_change_list = []
        for i in closing_change_list:
            sign_closing_change_list.append(sgn_0(i))
        obv_list = list(map(lambda x: x[0]*x[1], zip(sign_closing_change_list, item[4][1:])))
        raw_obv = sqrt_abs(sum(obv_list))
        OBV = float('%.3f' % raw_obv)
        stock_feature_dict["OBV"] = OBV
    
    
    
    
    
        exp_len = 49
        exp_starting = len(item[3]) - exp_len
        price_list_50 = [mean(item[3][:exp_starting])] + item[3][-exp_len:]
    
    
        const_50 = 2/51
        EMA_50 = []
        for i in range(1,len(price_list_50)):
            raw_ema_50 = const_50*price_list_50[i-1] + (1-const_50)*price_list_50[i]
            ema_50 = float('%.3f' % raw_ema_50)
            EMA_50.append(ema_50)
    
        const_25 = 2/26
        EMA_25 = []
        for i in range(26,len(price_list_50)):
            raw_ema_25 = const_25*price_list_50[i-1] + (1-const_25)*price_list_50[i]
            ema_25 = float('%.3f' % raw_ema_25)
            EMA_25.append(ema_25)
        
        EMA_50c = EMA_50[-24:]
        EMA_diff = list(map(lambda x: x[0]-x[1], zip(EMA_50c, EMA_25)))
        EMA_mean = np.mean(EMA_diff)*100
        EMA_diff_mean = float('%.5f' % np.mean(EMA_mean))
        stock_feature_dict['EMA_DIFF'] = EMA_diff_mean
        stock_feature_dict['EMA_TREND'] = sgn_num(EMA_diff)
        stock_feature_dict['EMA_OVERALL'] = sgn_total(EMA_diff)
    
    
    
    
    
        cr_pos = []
        cr_neg = []
    
        middle_price_list_c = item[7][:-1]
        closing_price_list_c = item[3][1:]
        cr_list = list(map(lambda x: x[0]-x[1], zip(middle_price_list_c, closing_price_list_c)))
        for i in cr_list:
            if i > 0:
                cr_pos.append(i)
            else:
                cr_neg.append(abs(i))
    
        cr_pos_sum = float('%.3f' % sum(cr_pos))
        cr_neg_sum = float('%.3f' % sum(cr_neg))
        raw_cr = 100*(cr_pos_sum/cr_neg_sum)
        CR = float('%.3f' % raw_cr)
        stock_feature_dict["CR"] = CR
    
    
    
        square_sum = []
        MA = mean(item[3])
        MB = mean(item[3][:-1])
    
        for i in item[3]:
            square_sum.append((i-MA)**2)
        MD = math.sqrt(mean(square_sum))
    
        raw_UP = MB+2*MD
        raw_DN = MB-2*MD
        UP = float('%.3f' % raw_UP)
        DN = float('%.3f' % raw_DN)
        stock_feature_dict["UP"] = UP
        stock_feature_dict["DN"] = DN
        
        
        if item[3][-1] > item[3][0]:
            stock_feature_dict["change_c"] = 1
        else:
            stock_feature_dict["change_c"] = -1
            
            
        stock_feature_list = list(stock_feature_dict.values())
        stock_feature_list = [-100 if math.isnan(x) else x for x in stock_feature_list]
        stock_feature_list = [100 if math.isinf(x) else x for x in stock_feature_list]
        
        
        training_data = stock_feature_list[:-1]
        training_data_total.append(training_data)
        
        training_label = stock_feature_list[-1]
        training_label_total.append(training_label)
        

print (len(training_data_total))


end = time.clock()
print ((end-start)/60, "min")


/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: 'U' mode is deprecated
/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:172: RuntimeWarning: divide by zero encountered in double_scalars
/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:161: RuntimeWarning: overflow encountered in exp
/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:172: RuntimeWarning: invalid value encountered in double_scalars
/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:156: RuntimeWarning: divide by zero encountered in double_scalars
/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:156: RuntimeWarning: invalid value encountered in double_scalars


51150
19.860666666666663 min


In [7]:
transformer = RobustScaler().fit(training_data_total)
data = transformer.transform(training_data_total)
print (len(data))

51150


# build the validation model 5 DAYS

In [8]:
start = time.clock()

validation_data_total = []
validation_label_total = []

for file in valid_file:
    
    open_price = []
    high_price = []
    low_price = []
    close_price = []
    volume = []
    change = []
    typical_price = []
    middle_price = []
    
    
    
    
    file_name = path + file
    
    with open(file_name, "rU", encoding = "ISO-8859-1") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:            
            
            diff = float(row[4]) - float(row[1])
            tp = (float(row[2]) + float(row[3]) + float(row[4]))/3
            mp = (float(row[2]) + float(row[3]))/2
        
            # tp = typical price, mp = middle price
            
            row.append(float('%.3f' % diff))
            row.append(float('%.3f' % tp))
            row.append(float('%.3f' % mp))
        
        
            open_price.append(float(row[1]))
            open_price_list = open_price[-700:]
            open_price_list_split_5 = slic(open_price_list, 9, 28)
            
            high_price.append(float(row[2]))
            high_price_list = high_price[-700:]
            high_price_list_split_5 = slic(high_price_list, 9, 28)
            
            low_price.append(float(row[3]))
            low_price_list = low_price[-700:]
            low_price_list_split_5 = slic(low_price_list, 9, 28)
            
            close_price.append(float(row[4]))
            close_price_list = close_price[-700:]
            close_price_list_split_5 = slic(close_price_list, 9, 28)
            
            volume.append(float(row[5]))
            volume_list = volume[-700:]
            volume_list_split_5 = slic(volume_list, 9, 28)
        
            change.append(row[7])
            change_list = change[-700:]
            change_list_split_5 = slic(change_list, 9, 28)
            
            typical_price.append(row[8])
            typical_price_list = typical_price[-700:]
            typical_price_list_split_5 = slic(typical_price_list, 9, 28)
            
            middle_price.append(row[9])
            middle_price_list = middle_price[-700:]
            middle_price_list_split_5 = slic(middle_price_list, 9, 28)
    
    validation_list = [open_price_list_split_5, high_price_list_split_5, low_price_list_split_5, close_price_list_split_5, \
                    volume_list_split_5, change_list_split_5, typical_price_list_split_5, middle_price_list_split_5]

    validation_list_mixed = mix(validation_list)

    
    
    
    stock_feature_dict = {}
    
    for item in validation_list_mixed:
    
        pos_list = []
        abs_list = []
    
        for flt in item[5]:
            abs_list.append(abs(flt))
            if flt > 0:
                pos_list.append(flt)
    
        abs_sum = float('%.3f' % sum(abs_list))
        pos_sum = float('%.3f' % sum(pos_list))
        raw_rsi = (pos_sum/(1+abs_sum))*100
        RSI = float('%.3f' % raw_rsi)
        stock_feature_dict['RSI'] = RSI 
        
        
        
        
        indicator_list = [0.5] 
    
        money_flow_list = [vol*tp for vol, tp in zip(item[4], item[6])]
        total_money_flow = float('%.2f' % sum(money_flow_list))
    
        for i in range(len(item[6])-1):
            det = sgn(float('%.2f' % (item[6][i+1] - item[6][i])))
            indicator_list.append(det)
       
        positive_money_flow_list = [ind*mf for ind, mf in zip(indicator_list, money_flow_list)]
        positive_money_flow = float('%.2f' % sum(positive_money_flow_list))
        negative_money_flow = total_money_flow - positive_money_flow
        money_rate = (positive_money_flow/(negative_money_flow+1))
    
        raw_mfi = 100-100/(1+money_rate)
        MFI = float('%.3f' % raw_mfi)
        stock_feature_dict['MFI'] = MFI   
    
    
        if len(item[3]) >= 3:
            raw_rsv = 100*(close_price_list[-1] - min(close_price_list))/(max(close_price_list) - min(close_price_list))
        RSV = float('%.3f' % raw_rsv)
        stock_feature_dict['RSV'] = RSV
    
    
    
        ax = item[3][-1] - item[3][-5]
        bx = item[3][-5]
        raw_roc = 100*ax/bx
        ROC = float('%.3f' % raw_roc)
        stock_feature_dict['ROC'] = ROC
    
    
        square_sum_5 = []
        TP_5 = mean(item[6][-5:])
        MA_5 = mean(item[3][-5:])
        for i in item[3][-5:]:
            square_sum_5.append((i-MA_5)**2)
        MD_5 = math.sqrt(mean(square_sum_5))
    
        raw_cci = (TP_5 - MA_5)/(MD_5*0.015)
        CCI = float('%.3f' % raw_cci)
        stock_feature_dict["CCI"] = CCI
    
    
        vol_para = list(map(lambda x: (2*x[0]-x[1]-x[2])/exp(x[1]-x[2]), zip(item[3], item[1], item[2])))
    
        va = []
        va_change_list = []
        va.append(item[4][0])
    
        for i in range(0, len(item[4])-1):
            va.append(va[i] + vol_para[i]*item[4][i+1])
    
        for i in range(0, len(va)-1):
            va_change_list.append(va[i+1] - va[i])
        va_change_rate = (va[-1] - va[0])/va[0]
    
        if abs(va_change_rate) > 0.1:
            VA = sgn(va_change_rate)
        else:
            VA = sgn_num(va_change_list)
        stock_feature_dict["VA"] = VA
    
    
    
    
        closing_change_list = []

        for i in range(0, len(item[3])-1):
            closing_change_list.append(item[3][i+1]-item[3][i])
   
        closing_price_list_pvt = item[3][1:]
        volume_list_pvt = volume_list[1:]
        pvt_list = list(map(lambda x: x[0]*x[1]/x[2], zip(closing_change_list, volume_list_pvt, closing_price_list_pvt)))
        raw_pvt = sqrt_abs(sum(pvt_list))
        PVT = float('%.3f' % raw_pvt)
        stock_feature_dict["PVT"] = PVT
    
    
    
    
        sign_closing_change_list = []
        for i in closing_change_list:
            sign_closing_change_list.append(sgn_0(i))
        obv_list = list(map(lambda x: x[0]*x[1], zip(sign_closing_change_list, item[4][1:])))
        raw_obv = sqrt_abs(sum(obv_list))
        OBV = float('%.3f' % raw_obv)
        stock_feature_dict["OBV"] = OBV
    
    
    
    
    
        exp_len = 4
        exp_starting = len(item[3]) - exp_len
        price_list_50 = [mean(item[3][:exp_starting])] + item[3][-exp_len:]
    
    
        const_50 = 2/6
        EMA_50 = []
        for i in range(1,len(price_list_50)):
            raw_ema_50 = const_50*price_list_50[i-1] + (1-const_50)*price_list_50[i]
            ema_50 = float('%.3f' % raw_ema_50)
            EMA_50.append(ema_50)
    
        const_25 = 2/3
        EMA_25 = []
        for i in range(3,len(price_list_50)):
            raw_ema_25 = const_25*price_list_50[i-1] + (1-const_25)*price_list_50[i]
            ema_25 = float('%.3f' % raw_ema_25)
            EMA_25.append(ema_25)
        
        EMA_50c = EMA_50[-2:]
        EMA_diff = list(map(lambda x: x[0]-x[1], zip(EMA_50c, EMA_25)))
        EMA_mean = np.mean(EMA_diff)*100
        EMA_diff_mean = float('%.5f' % np.mean(EMA_mean))
        stock_feature_dict['EMA_DIFF'] = EMA_diff_mean
        stock_feature_dict['EMA_TREND'] = sgn_num(EMA_diff)
        stock_feature_dict['EMA_OVERALL'] = sgn_total(EMA_diff)
    
    
    
    
    
        cr_pos = []
        cr_neg = []
    
        middle_price_list_c = item[7][:-1]
        closing_price_list_c = item[3][1:]
        cr_list = list(map(lambda x: x[0]-x[1], zip(middle_price_list_c, closing_price_list_c)))
        for i in cr_list:
            if i > 0:
                cr_pos.append(i)
            else:
                cr_neg.append(abs(i))
    
        cr_pos_sum = float('%.3f' % sum(cr_pos))
        cr_neg_sum = float('%.3f' % sum(cr_neg))
        raw_cr = 100*(cr_pos_sum/(1+cr_neg_sum))
        CR = float('%.3f' % raw_cr)
        stock_feature_dict["CR"] = CR
    
    
    
        square_sum = []
        MA = mean(item[3])
        MB = mean(item[3][:-1])
    
        for i in item[3]:
            square_sum.append((i-MA)**2)
        MD = math.sqrt(mean(square_sum))
    
        raw_UP = MB+2*MD
        raw_DN = MB-2*MD
        UP = float('%.3f' % raw_UP)
        DN = float('%.3f' % raw_DN)
        stock_feature_dict["UP"] = UP
        stock_feature_dict["DN"] = DN
        
        
        if item[3][-1] > item[3][0]:
            stock_feature_dict["change_c"] = 1
        else:
            stock_feature_dict["change_c"] = -1
            
            
        stock_feature_list = list(stock_feature_dict.values())
        stock_feature_list = [-100 if math.isnan(x) else x for x in stock_feature_list]
        stock_feature_list = [100 if math.isinf(x) else x for x in stock_feature_list]
        
        
        validation_data = stock_feature_list[:-1]
        validation_data_total.append(validation_data)
        
        validation_label = stock_feature_list[-1]
        validation_label_total.append(validation_label)
        

print (len(validation_data_total))


end = time.clock()
print ((end-start)/60, "min")


/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: 'U' mode is deprecated
/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:155: RuntimeWarning: divide by zero encountered in double_scalars
/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:144: RuntimeWarning: overflow encountered in exp
/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:139: RuntimeWarning: divide by zero encountered in double_scalars
/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:155: RuntimeWarning: invalid value encountered in double_scalars
/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in double_scalars


46035
14.576833333333335 min


In [9]:
transformer_v = RobustScaler().fit(validation_data_total)
validation_data = transformer_v.transform(validation_data_total)

In [10]:
start = time.clock()

testing_data_total = []
testing_label_total = []
testing_list_total = []

count_term=1

for file in valid_file:
    
    open_price = []
    high_price = []
    low_price = []
    close_price = []
    volume = []
    change = []
    typical_price = []
    middle_price = []
    
    
    
    
    file_name = path + file
    
    
    with open(file_name, "rU", encoding = "ISO-8859-1") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:            
            
            diff = float(row[4]) - float(row[1])
            tp = (float(row[2]) + float(row[3]) + float(row[4]))/3
            mp = (float(row[2]) + float(row[3]))/2
        
            # tp = typical price, mp = middle price
            
            row.append(float('%.3f' % diff))
            row.append(float('%.3f' % tp))
            row.append(float('%.3f' % mp))
        
        
            open_price.append(float(row[1]))
            open_price_list_testing = open_price[-100:-72]
        
        
            high_price.append(float(row[2]))
            high_price_list_testing = high_price[-100:-72]
            
            low_price.append(float(row[3]))
            low_price_list_testing = low_price[-100:-72]
            
            close_price.append(float(row[4]))
            close_price_list_testing = close_price[-100:-72]
            
            volume.append(float(row[5]))
            volume_list_testing = volume[-100:-72]
            
            change.append(row[7])
            change_list_testing = change[-100:-72]
            
            typical_price.append(row[8])
            typical_price_list_testing = typical_price[-100:-72]
            
            middle_price.append(row[9])
            middle_price_list_testing = middle_price[-100:-72]
    
    testing_list = [open_price_list_testing, high_price_list_testing, low_price_list_testing, close_price_list_testing, \
                    volume_list_testing, change_list_testing, typical_price_list_testing, middle_price_list_testing]


    
    
    
    stock_feature_dict = {}
    
    
    
    pos_list = []
    abs_list = []
    
    for flt in testing_list[5]:
        abs_list.append(abs(flt))
        if flt > 0:
            pos_list.append(flt)
    
    abs_sum = float('%.3f' % sum(abs_list))
    pos_sum = float('%.3f' % sum(pos_list))
    raw_rsi = (pos_sum/(1+abs_sum))*100
    RSI = float('%.3f' % raw_rsi)
    stock_feature_dict['RSI'] = RSI 
        
        
        
        
    indicator_list = [0.5] 
    
    money_flow_list = [vol*tp for vol, tp in zip(testing_list[4], testing_list[6])]
    total_money_flow = float('%.2f' % sum(money_flow_list))
    
    for i in range(len(testing_list[6])-1):
        det = sgn(float('%.2f' % (testing_list[6][i+1] - testing_list[6][i])))
        indicator_list.append(det)
       
    positive_money_flow_list = [ind*mf for ind, mf in zip(indicator_list, money_flow_list)]
    positive_money_flow = float('%.2f' % sum(positive_money_flow_list))
    negative_money_flow = total_money_flow - positive_money_flow
    money_rate = (positive_money_flow/negative_money_flow)
    
    raw_mfi = 100-100/(1+money_rate)
    MFI = float('%.3f' % raw_mfi)
    stock_feature_dict['MFI'] = MFI   
    
    
    if len(testing_list[3]) >= 3:
        raw_rsv = 100*(close_price_list_testing[-1] - min(close_price_list_testing))/(max(close_price_list_testing) - min(close_price_list_testing))
    RSV = float('%.3f' % raw_rsv)
    stock_feature_dict['RSV'] = RSV
    
    
    
    ax = testing_list[3][-1] - testing_list[3][-5]
    bx = testing_list[3][-5]
    raw_roc = 100*ax/bx
    ROC = float('%.3f' % raw_roc)
    stock_feature_dict['ROC'] = ROC
    
    
    square_sum_5 = []
    TP_5 = mean(testing_list[6][-5:])
    MA_5 = mean(testing_list[3][-5:])
    for i in testing_list[3][-5:]:
        square_sum_5.append((i-MA_5)**2)
        MD_5 = math.sqrt(mean(square_sum_5))
    
    raw_cci = (TP_5 - MA_5)/(MD_5*0.015)
    CCI = float('%.3f' % raw_cci)
    stock_feature_dict["CCI"] = CCI
    
    
    vol_para = list(map(lambda x: (2*x[0]-x[1]-x[2])/exp(x[1]-x[2]), zip(testing_list[3], testing_list[1], testing_list[2])))
    
    va = []
    va_change_list = []
    va.append(testing_list[4][0])
    
    for i in range(0, len(testing_list[4])-1):
        va.append(va[i] + vol_para[i]*testing_list[4][i+1])
    
    for i in range(0, len(va)-1):
        va_change_list.append(va[i+1] - va[i])
    va_change_rate = (va[-1] - va[0])/va[0]
    
    if abs(va_change_rate) > 0.1:
        VA = sgn(va_change_rate)
    else:
        VA = sgn_num(va_change_list)
    stock_feature_dict["VA"] = VA
    
    
    
    
    closing_change_list = []

    for i in range(0, len(testing_list[3])-1):
        closing_change_list.append(testing_list[3][i+1]-testing_list[3][i])
   
    closing_price_list_pvt = testing_list[3][1:]
    volume_list_pvt = volume_list_testing[1:]
    pvt_list = list(map(lambda x: x[0]*x[1]/x[2], zip(closing_change_list, volume_list_pvt, closing_price_list_pvt)))
    raw_pvt = sqrt_abs(sum(pvt_list))
    PVT = float('%.3f' % raw_pvt)
    stock_feature_dict["PVT"] = PVT
    
    
    
    
    sign_closing_change_list = []
    for i in closing_change_list:
        sign_closing_change_list.append(sgn_0(i))
    obv_list = list(map(lambda x: x[0]*x[1], zip(sign_closing_change_list, testing_list[4][1:])))
    raw_obv = sqrt_abs(sum(obv_list))
    OBV = float('%.3f' % raw_obv)
    stock_feature_dict["OBV"] = OBV
    
    
    
    
    
    exp_len = 49
    exp_starting = len(testing_list[3]) - exp_len
    price_list_50 = [mean(testing_list[3][:exp_starting])] + testing_list[3][-exp_len:]
    
    
    const_50 = 2/51
    EMA_50 = []
    for i in range(1,len(price_list_50)):
        raw_ema_50 = const_50*price_list_50[i-1] + (1-const_50)*price_list_50[i]
        ema_50 = float('%.3f' % raw_ema_50)
        EMA_50.append(ema_50)
    
    const_25 = 2/26
    EMA_25 = []
    for i in range(26,len(price_list_50)):
        raw_ema_25 = const_25*price_list_50[i-1] + (1-const_25)*price_list_50[i]
        ema_25 = float('%.3f' % raw_ema_25)
        EMA_25.append(ema_25)
        
    EMA_50c = EMA_50[-24:]
    EMA_diff = list(map(lambda x: x[0]-x[1], zip(EMA_50c, EMA_25)))
    EMA_mean = np.mean(EMA_diff)*100
    EMA_diff_mean = float('%.5f' % np.mean(EMA_mean))
    stock_feature_dict['EMA_DIFF'] = EMA_diff_mean
    stock_feature_dict['EMA_TREND'] = sgn_num(EMA_diff)
    stock_feature_dict['EMA_OVERALL'] = sgn_total(EMA_diff)
    
    
    
    
    
    cr_pos = []
    cr_neg = []
    
    middle_price_list_c = testing_list[7][:-1]
    closing_price_list_c = testing_list[3][1:]
    cr_list = list(map(lambda x: x[0]-x[1], zip(middle_price_list_c, closing_price_list_c)))
    for i in cr_list:
        if i > 0:
            cr_pos.append(i)
        else:
            cr_neg.append(abs(i))
    
    cr_pos_sum = float('%.3f' % sum(cr_pos))
    cr_neg_sum = float('%.3f' % sum(cr_neg))
    raw_cr = 100*(cr_pos_sum/cr_neg_sum)
    CR = float('%.3f' % raw_cr)
    stock_feature_dict["CR"] = CR
    
    
    
    square_sum = []
    MA = mean(testing_list[3])
    MB = mean(testing_list[3][:-1])
    
    for i in testing_list[3]:
        square_sum.append((i-MA)**2)
    MD = math.sqrt(mean(square_sum))
    
    raw_UP = MB+2*MD
    raw_DN = MB-2*MD
    UP = float('%.3f' % raw_UP)
    DN = float('%.3f' % raw_DN)
    stock_feature_dict["UP"] = UP
    stock_feature_dict["DN"] = DN
        
        
        
    if testing_list[3][-1] > testing_list[3][0]:
        stock_feature_dict["change_c"] = 1
    else:
        stock_feature_dict["change_c"] = -1
            
            
    stock_feature_list = list(stock_feature_dict.values())
    stock_feature_list = [-100 if math.isnan(x) else x for x in stock_feature_list]
    stock_feature_list = [100 if math.isinf(x) else x for x in stock_feature_list]
        
        
    testing_data = stock_feature_list[:-1]
    testing_data_total.append(testing_data)
        
    testing_label = stock_feature_list[-1]
    testing_label_total.append(testing_label)
        
    print (file)
    print (count_term)
    count_term += 1
    
    
print (len(testing_list_total))


end = time.clock()
print ((end-start)/60, "min")


/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: 'U' mode is deprecated


epe.us.csv
1
cr.us.csv
2
arw.us.csv
3
hta.us.csv
4
syy.us.csv
5
rmti.us.csv
6
luna.us.csv
7
wms.us.csv
8
ktec.us.csv
9
car.us.csv
10
zen.us.csv
11
ghc.us.csv
12
clne.us.csv
13
mxwl.us.csv
14
ti-a.us.csv
15
utf.us.csv
16
cplp.us.csv
17
cy.us.csv
18
tcpc.us.csv
19
seneb.us.csv
20
gfi.us.csv
21
ascma.us.csv
22
quad.us.csv
23
hbnc.us.csv
24
tg.us.csv
25
fcn.us.csv
26
slg_i.us.csv
27
phx.us.csv
28
dfs_b.us.csv
29
skis.us.csv
30
pho.us.csv
31
tcf.us.csv
32
scm.us.csv
33
ltbr.us.csv
34
banr.us.csv
35
slm.us.csv
36
iphi.us.csv
37
san.us.csv
38
csv.us.csv
39
mxim.us.csv
40
rox.us.csv
41
wf.us.csv
42
cphi.us.csv
43
earn.us.csv
44
pei_b.us.csv
45
lyg.us.csv
46
ndp.us.csv
47
arh_c.us.csv
48
rushb.us.csv
49
sgf.us.csv
50
wwd.us.csv
51
esrt.us.csv
52
itic.us.csv
53
cprt.us.csv
54
irt.us.csv
55
awi.us.csv
56
tfx.us.csv
57
esxb.us.csv
58
tsla.us.csv
59
spwh.us.csv
60
crt.us.csv
61
jd.us.csv
62
xpl.us.csv
63
mmm.us.csv
64
emcf.us.csv
65
tvpt.us.csv
66
vrtx.us.csv
67
gjv.us.csv
68
mcv.us.csv
69
jtd.us.c

/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:140: RuntimeWarning: overflow encountered in exp


ovbc.us.csv
138
umh.us.csv
139
dhg.us.csv
140
cys_a.us.csv
141
rlj.us.csv
142
fsam.us.csv
143
hmn.us.csv
144
brt.us.csv
145
expr.us.csv
146
asml.us.csv
147
gs_b.us.csv
148
ab.us.csv
149
gbnk.us.csv
150
psa_v.us.csv
151
glp.us.csv
152
drd.us.csv
153
wmb.us.csv
154
occ.us.csv
155
adp.us.csv
156
vlp.us.csv
157
wft.us.csv
158
towr.us.csv
159
dpg.us.csv
160
tlrd.us.csv
161
adbe.us.csv
162
rgls.us.csv
163
erj.us.csv
164
pbip.us.csv
165
eae.us.csv
166
obci.us.csv
167
vrtu.us.csv
168
bsac.us.csv
169
ocrx.us.csv
170
ubnk.us.csv
171
mnk.us.csv
172
mtrn.us.csv
173
comm.us.csv
174
reis.us.csv
175
swk.us.csv
176
hvt-a.us.csv
177
caf.us.csv
178
pei.us.csv
179
lzb.us.csv
180
cf.us.csv
181
ccne.us.csv
182
homb.us.csv
183
pfs.us.csv
184
vslr.us.csv
185
aig-ws.us.csv
186
ypf.us.csv
187
cube.us.csv
188
gnbc.us.csv
189
hrg.us.csv
190
chdn.us.csv
191
vvus.us.csv
192
awf.us.csv
193
kio.us.csv
194
brkr.us.csv
195
play.us.csv
196
ljpc.us.csv
197
nrcib.us.csv
198
b.us.csv
199
tnp.us.csv
200
brkl.us.csv
201
jrj

/local/java/python-pip-packages.cs909/lib/python3.6/site-packages/ipykernel_launcher.py:135: RuntimeWarning: divide by zero encountered in double_scalars


...

mtg.us.csv
2867
apa.us.csv
2868
fun.us.csv
2869
lea.us.csv
2870
ste.us.csv
2871
rig.us.csv
2872
lbai.us.csv
2873
rds-b.us.csv
2874
schw.us.csv
2875
strs.us.csv
2876
gild.us.csv
2877
micr.us.csv
2878
cban.us.csv
2879
ger.us.csv
2880
rds-a.us.csv
2881
cqh.us.csv
2882
ptie.us.csv
2883
axti.us.csv
2884
scmp.us.csv
2885
xl.us.csv
2886
bbt_d.us.csv
2887
bml_j.us.csv
2888
crd-a.us.csv
2889
trk.us.csv
2890
amn.us.csv
2891
kyn_f.us.csv
2892
boe.us.csv
2893
arql.us.csv
2894
wrb_b.us.csv
2895
reph.us.csv
2896
res.us.csv
2897
spab.us.csv
2898
fcx.us.csv
2899
xhr.us.csv
2900
sdrl.us.csv
2901
bvsn.us.csv
2902
astc.us.csv
2903
cit.us.csv
2904
tlra.us.csv
2905
hmg.us.csv
2906
gef-b.us.csv
2907
mfl.us.csv
2908
grp-u.us.csv
2909
mfc.us.csv
2910
mdgl.us.csv
2911
pegi.us.csv
2912
geb.us.csv
2913
sofo.us.csv
2914
cvs.us.csv
2915
m.us.csv
2916
bcr.us.csv
2917
glo.us.csv
2918
holx.us.csv
2919
xray.us.csv
2920
whlrp.us.csv
2921
agm_a.us.csv
2922
stb.us.csv
2923
ajg.us.csv
2924
lcm.us.csv
2925
dsw.us.csv
2926


gs_c.us.csv
3366
town.us.csv
3367
rso_c.us.csv
3368
rli.us.csv
3369
ctl.us.csv
3370
umpq.us.csv
3371
rrgb.us.csv
3372
yuma.us.csv
3373
tlgt.us.csv
3374
usdp.us.csv
3375
cavm.us.csv
3376
crl.us.csv
3377
ed.us.csv
3378
rex.us.csv
3379
spdw.us.csv
3380
ohrp.us.csv
3381
ston.us.csv
3382
noc.us.csv
3383
luk.us.csv
3384
rtrx.us.csv
3385
hurn.us.csv
3386
ggb.us.csv
3387
fcfs.us.csv
3388
aon.us.csv
3389
fra.us.csv
3390
nq.us.csv
3391
aer.us.csv
3392
dyn_a.us.csv
3393
rlh.us.csv
3394
klxi.us.csv
3395
ctlt.us.csv
3396
wbil.us.csv
3397
yndx.us.csv
3398
ahgp.us.csv
3399
dxr.us.csv
3400
cpt.us.csv
3401
fhn.us.csv
3402
zais.us.csv
3403
btg.us.csv
3404
sbs.us.csv
3405
mgic.us.csv
3406
btn.us.csv
3407
iif.us.csv
3408
mos.us.csv
3409
cns.us.csv
3410
umbf.us.csv
3411
txrh.us.csv
3412
slp.us.csv
3413
mpx.us.csv
3414
mcr.us.csv
3415
vac.us.csv
3416
wilc.us.csv
3417
dy.us.csv
3418
mpwr.us.csv
3419
nid.us.csv
3420
pkx.us.csv
3421
emf.us.csv
3422
pf.us.csv
3423
hty.us.csv
3424
msl.us.csv
3425
splb.us.csv
342

exas.us.csv
3863
orrf.us.csv
3864
igc.us.csv
3865
mqy.us.csv
3866
zgnx.us.csv
3867
ide.us.csv
3868
spa.us.csv
3869
airt.us.csv
3870
mcri.us.csv
3871
ango.us.csv
3872
pii.us.csv
3873
aey.us.csv
3874
silc.us.csv
3875
el.us.csv
3876
tech.us.csv
3877
cri.us.csv
3878
lpsn.us.csv
3879
sti.us.csv
3880
glu.us.csv
3881
pdli.us.csv
3882
dni.us.csv
3883
pco.us.csv
3884
tmo.us.csv
3885
kro.us.csv
3886
glbz.us.csv
3887
buse.us.csv
3888
rs.us.csv
3889
sfs.us.csv
3890
fox.us.csv
3891
mlco.us.csv
3892
thr.us.csv
3893
celp.us.csv
3894
gs.us.csv
3895
rvt.us.csv
3896
gigm.us.csv
3897
fb.us.csv
3898
ajx.us.csv
3899
ait.us.csv
3900
egov.us.csv
3901
wlfc.us.csv
3902
rnn.us.csv
3903
wair.us.csv
3904
vtrb.us.csv
3905
alco.us.csv
3906
mgf.us.csv
3907
mth.us.csv
3908
ew.us.csv
3909
vbiv.us.csv
3910
usb.us.csv
3911
ngd.us.csv
3912
osbc.us.csv
3913
nvgn.us.csv
3914
dysl.us.csv
3915
rexr.us.csv
3916
pm.us.csv
3917
amh.us.csv
3918
tkc.us.csv
3919
xin.us.csv
3920
panw.us.csv
3921
mov.us.csv
3922
seac.us.csv
3923
rns

bioa.us.csv
4360
ppx.us.csv
4361
becn.us.csv
4362
eurn.us.csv
4363
stly.us.csv
4364
hcn_i.us.csv
4365
nsu.us.csv
4366
unam.us.csv
4367
eigi.us.csv
4368
wfc_p.us.csv
4369
ardc.us.csv
4370
cree.us.csv
4371
eqr.us.csv
4372
hive.us.csv
4373
valx.us.csv
4374
anik.us.csv
4375
pfl.us.csv
4376
csf.us.csv
4377
irmd.us.csv
4378
mplx.us.csv
4379
vrtv.us.csv
4380
llnw.us.csv
4381
akam.us.csv
4382
orc.us.csv
4383
cato.us.csv
4384
lake.us.csv
4385
mndo.us.csv
4386
prkr.us.csv
4387
gpic.us.csv
4388
hdb.us.csv
4389
bif.us.csv
4390
kfy.us.csv
4391
arcx.us.csv
4392
pfpt.us.csv
4393
ovas.us.csv
4394
cb.us.csv
4395
an.us.csv
4396
jrvr.us.csv
4397
sci.us.csv
4398
erh.us.csv
4399
sptl.us.csv
4400
lly.us.csv
4401
ntri.us.csv
4402
ziop.us.csv
4403
kmg.us.csv
4404
ambcw.us.csv
4405
gyro.us.csv
4406
cxe.us.csv
4407
sppp.us.csv
4408
dlx.us.csv
4409
pnf.us.csv
4410
spem.us.csv
4411
unf.us.csv
4412
ual.us.csv
4413
alsk.us.csv
4414
ghyg.us.csv
4415
eqt.us.csv
4416
mrcy.us.csv
4417
jbl.us.csv
4418
oclr.us.csv
4419
s

cbt.us.csv
4856
wtm.us.csv
4857
hmlp.us.csv
4858
xyl.us.csv
4859
snv_c.us.csv
4860
fran.us.csv
4861
wfc_l.us.csv
4862
goodo.us.csv
4863
bcc.us.csv
4864
xpo.us.csv
4865
nkx.us.csv
4866
novt.us.csv
4867
phg.us.csv
4868
vc.us.csv
4869
ulh.us.csv
4870
tef.us.csv
4871
atrs.us.csv
4872
htld.us.csv
4873
ctg.us.csv
4874
nhi.us.csv
4875
ffg.us.csv
4876
impv.us.csv
4877
esl.us.csv
4878
amkr.us.csv
4879
cs.us.csv
4880
jqc.us.csv
4881
sti_e.us.csv
4882
leg.us.csv
4883
seed.us.csv
4884
stm.us.csv
4885
dvax.us.csv
4886
chscp.us.csv
4887
eto.us.csv
4888
salm.us.csv
4889
dnp.us.csv
4890
lq.us.csv
4891
sjr.us.csv
4892
prmw.us.csv
4893
club.us.csv
4894
dltr.us.csv
4895
rrts.us.csv
4896
rf.us.csv
4897
adxs.us.csv
4898
srlp.us.csv
4899
has.us.csv
4900
vgr.us.csv
4901
dlhc.us.csv
4902
pdco.us.csv
4903
fold.us.csv
4904
ffkt.us.csv
4905
bml_g.us.csv
4906
gjs.us.csv
4907
cizn.us.csv
4908
trst.us.csv
4909
ulta.us.csv
4910
gff.us.csv
4911
staa.us.csv
4912
ovly.us.csv
4913
gst_b.us.csv
4914
wex.us.csv
4915
ensg.

In [11]:
transformer_testing = RobustScaler().fit(testing_data_total)
data_testing = transformer_testing.transform(testing_data_total)

In [12]:

start = time.clock()

dt_val_acc = []
dt_testing_acc = []

log_val_acc = []
log_testing_acc = []

nb_val_acc = []
nb_testing_acc = []

svm_val_acc = []
svm_testing_acc = []

rf_val_acc = []
rf_testing_acc = []


training_data_split = cut(data, 9)
training_label_split = cut(training_label_total, 9)

validation_data_split = cut(validation_data, 9)
validation_label_split = cut(validation_label_total, 9)

# print (training_data_split[2])

for i in range (0, 9):
    training_sub_list = []
    training_sub_list.append(training_data_split[i])
    training_sub_list.append(training_label_split[i])
    
    validation_sub_list = []
    validation_sub_list.append(validation_data_split[i])
    validation_sub_list.append(validation_label_split[i])
    
    
    
    model1 = tree.DecisionTreeClassifier()
    dt_model = model1.fit(training_sub_list[0], training_sub_list[-1])
    dt_result = dt_model.predict(validation_sub_list[0])
    dt_testing_result = dt_model.predict(testing_data_total)
    
    print ("decision tree has accuracy on validation", accuracy_score(validation_sub_list[-1], dt_result))
    dt_val_acc.append(float(format(accuracy_score(validation_sub_list[-1], dt_result), '.3f')))
    
    print ("decision tree has precision on validation", precision_score(validation_sub_list[-1], dt_result))
    print ("decision tree has recall on validation", recall_score(validation_sub_list[-1], dt_result))
    print (classification_report(validation_sub_list[-1], dt_result))
    print (confusion_matrix(validation_sub_list[-1], dt_result))
    print ("===========================")
    print ("decision tree has accuracy on testing", accuracy_score(testing_label_total, dt_testing_result))
    dt_testing_acc.append(float(format(accuracy_score(testing_label_total, dt_testing_result), '.3f')))
    
    print ("decision tree has precision on testing", precision_score(testing_label_total, dt_testing_result))
    print ("decision tree has recall on testing", recall_score(testing_label_total, dt_testing_result))
    print (classification_report(testing_label_total, dt_testing_result))
    print (confusion_matrix(testing_label_total, dt_testing_result))    
    print ("============================================================")
    
    
    
    model2 = LogisticRegression(random_state=0, solver='liblinear', multi_class='ovr')
    log_model = model2.fit(training_sub_list[0], training_sub_list[-1])
    log_result = log_model.predict(validation_sub_list[0])
    log_testing_result = log_model.predict(testing_data_total)
    
    print ("log reg has accuracy on validation", accuracy_score(validation_sub_list[-1], log_result))
    log_val_acc.append(float(format(accuracy_score(validation_sub_list[-1], log_result), '.3f')))
    
    print ("log reg has precision on validation", precision_score(validation_sub_list[-1], log_result))
    print ("log reg has recall on validation", recall_score(validation_sub_list[-1], log_result))
    print (classification_report(validation_sub_list[-1], log_result))
    print (confusion_matrix(validation_sub_list[-1], log_result))
    print ("===========================")
    print ("log reg has accuracy on testing", accuracy_score(testing_label_total, log_testing_result))
    log_testing_acc.append(float(format(accuracy_score(testing_label_total, log_testing_result), '.3f')))
    
    print ("log reg has precision on testing", precision_score(testing_label_total, log_testing_result))
    print ("log reg has recall on testing", recall_score(testing_label_total, log_testing_result))
    print (classification_report(testing_label_total, log_testing_result))
    print (confusion_matrix(testing_label_total, log_testing_result)) 
    print ("============================================================")
   

    
    model3 = GaussianNB()
    nb_model = model3.fit(training_sub_list[0], training_sub_list[-1])
    nb_result = nb_model.predict(validation_sub_list[0])
    nb_testing_result = nb_model.predict(testing_data_total)
    
    print ("naive bayes has accuracy on validation", accuracy_score(validation_sub_list[-1], nb_result))
    nb_val_acc.append(float(format(accuracy_score(validation_sub_list[-1], nb_result), '.3f')))
    
    print ("naive bayes has precision on validation", precision_score(validation_sub_list[-1], nb_result))
    print ("naive bayes has recall on validation", recall_score(validation_sub_list[-1], nb_result))
    print (classification_report(validation_sub_list[-1], nb_result))
    print (confusion_matrix(validation_sub_list[-1], nb_result))
    print ("===========================")
    print ("naive bayes has accuracy on testing", accuracy_score(testing_label_total, nb_testing_result))
    nb_testing_acc.append(float(format(accuracy_score(testing_label_total, nb_testing_result), '.3f')))
    
    print ("naive bayes has precision on testing", precision_score(testing_label_total, nb_testing_result))
    print ("naive bayes has recall on testing", recall_score(testing_label_total, nb_testing_result))
    print (classification_report(testing_label_total, nb_testing_result))
    print (confusion_matrix(testing_label_total, nb_testing_result)) 
    print ("============================================================")
    
    
    
    model4 = LinearSVC(random_state=0, tol=1e-5)
    svm_model = model4.fit(training_sub_list[0], training_sub_list[-1])
    svm_result = svm_model.predict(validation_sub_list[0])
    svm_testing_result = svm_model.predict(testing_data_total)
    
    print ("svm has accuracy on validation", accuracy_score(validation_sub_list[-1], svm_result))
    svm_val_acc.append(float(format(accuracy_score(validation_sub_list[-1], svm_result), '.3f')))
    
    print ("svm has precision on validation", precision_score(validation_sub_list[-1], svm_result))
    print ("svm has recall on validation", recall_score(validation_sub_list[-1], svm_result))
    print (classification_report(validation_sub_list[-1], svm_result))
    print (confusion_matrix(validation_sub_list[-1], svm_result))
    print ("===========================")
    print ("svm has accuracy on testing", accuracy_score(testing_label_total, svm_testing_result))
    svm_testing_acc.append(float(format(accuracy_score(testing_label_total, svm_testing_result), '.3f')))
    
    print ("svm has precision on testing", precision_score(testing_label_total, svm_testing_result))
    print ("svm has recall on testing", recall_score(testing_label_total, svm_testing_result))
    print (classification_report(testing_label_total, svm_testing_result))
    print (confusion_matrix(testing_label_total, svm_testing_result)) 
    print ("============================================================")
    
    
    
    model5 = RandomForestClassifier(n_estimators=500, max_depth=2, random_state=0)
    rf_model = model5.fit(training_sub_list[0], training_sub_list[-1])
    rf_result = rf_model.predict(validation_sub_list[0])
    rf_testing_result = rf_model.predict(testing_data_total)
    
    print ("random forest has accuracy on validation", accuracy_score(validation_sub_list[-1], rf_result))
    rf_val_acc.append(float(format(accuracy_score(validation_sub_list[-1], rf_result), '.3f')))
    
    print ("random forest has precision on validation", precision_score(validation_sub_list[-1], rf_result))
    print ("random forest has recall on validation", recall_score(validation_sub_list[-1], rf_result))
    print (classification_report(validation_sub_list[-1], rf_result))
    print (confusion_matrix(validation_sub_list[-1], rf_result))
    print ("===========================")
    print ("random forest has accuracy on testing", accuracy_score(testing_label_total, rf_testing_result))
    rf_testing_acc.append(float(format(accuracy_score(testing_label_total, rf_testing_result), '.3f')))
    
    print ("random forest has precision on testing", precision_score(testing_label_total, rf_testing_result))
    print ("random forest has recall on testing", recall_score(testing_label_total, rf_testing_result))
    print (classification_report(testing_label_total, rf_testing_result))
    print (confusion_matrix(testing_label_total, rf_testing_result)) 
    print ("============================================================")
    
    

    
end = time.clock()
print ((end-start), "sec")

decision tree has accuracy on validation 0.8001955034213099
decision tree has precision on validation 0.7683383233532934
decision tree has recall on validation 0.8359120521172638
              precision    recall  f1-score   support

          -1       0.84      0.77      0.80      2659
           1       0.77      0.84      0.80      2456

   micro avg       0.80      0.80      0.80      5115
   macro avg       0.80      0.80      0.80      5115
weighted avg       0.80      0.80      0.80      5115

[[2040  619]
 [ 403 2053]]
decision tree has accuracy on testing 0.24574780058651027
decision tree has precision on testing 0.33849969001859886
decision tree has recall on testing 0.16362001798022174
              precision    recall  f1-score   support

          -1       0.20      0.40      0.27      1778
           1       0.34      0.16      0.22      3337

   micro avg       0.25      0.25      0.25      5115
   macro avg       0.27      0.28      0.24      5115
weighted avg       0.2

/local/java/python-pip-packages.cs909/lib64/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


svm has accuracy on validation 0.8189638318670577
svm has precision on validation 0.7610921501706485
svm has recall on validation 0.9079804560260586
              precision    recall  f1-score   support

          -1       0.90      0.74      0.81      2659
           1       0.76      0.91      0.83      2456

   micro avg       0.82      0.82      0.82      5115
   macro avg       0.83      0.82      0.82      5115
weighted avg       0.83      0.82      0.82      5115

[[1959  700]
 [ 226 2230]]
svm has accuracy on testing 0.34760508308895405
svm has precision on testing 0.5
svm has recall on testing 0.00029967036260113877
              precision    recall  f1-score   support

          -1       0.35      1.00      0.52      1778
           1       0.50      0.00      0.00      3337

   micro avg       0.35      0.35      0.35      5115
   macro avg       0.42      0.50      0.26      5115
weighted avg       0.45      0.35      0.18      5115

[[1777    1]
 [3336    1]]
random forest

/local/java/python-pip-packages.cs909/lib64/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


svm has accuracy on validation 0.8510263929618769
svm has precision on validation 0.6595469255663431
svm has recall on validation 0.8119521912350598
              precision    recall  f1-score   support

          -1       0.93      0.86      0.90      3860
           1       0.66      0.81      0.73      1255

   micro avg       0.85      0.85      0.85      5115
   macro avg       0.80      0.84      0.81      5115
weighted avg       0.87      0.85      0.86      5115

[[3334  526]
 [ 236 1019]]
svm has accuracy on testing 0.34760508308895405
svm has precision on testing 0.5
svm has recall on testing 0.00029967036260113877
              precision    recall  f1-score   support

          -1       0.35      1.00      0.52      1778
           1       0.50      0.00      0.00      3337

   micro avg       0.35      0.35      0.35      5115
   macro avg       0.42      0.50      0.26      5115
weighted avg       0.45      0.35      0.18      5115

[[1777    1]
 [3336    1]]
random forest

...

svm has accuracy on validation 0.884457478005865
svm has precision on validation 0.8955484434099505
svm has recall on validation 0.9299093655589123
              precision    recall  f1-score   support

          -1       0.86      0.80      0.83      1805
           1       0.90      0.93      0.91      3310

   micro avg       0.88      0.88      0.88      5115
   macro avg       0.88      0.87      0.87      5115
weighted avg       0.88      0.88      0.88      5115

[[1446  359]
 [ 232 3078]]
svm has accuracy on testing 0.3485826001955034
svm has precision on testing 0.8571428571428571
svm has recall on testing 0.0017980221756068325
              precision    recall  f1-score   support

          -1       0.35      1.00      0.52      1778
           1       0.86      0.00      0.00      3337

   micro avg       0.35      0.35      0.35      5115
   macro avg       0.60      0.50      0.26      5115
weighted avg       0.68      0.35      0.18      5115

[[1777    1]
 [3331    6]]
r

/local/java/python-pip-packages.cs909/lib64/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/local/java/python-pip-packages.cs909/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/local/java/python-pip-packages.cs909/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


log reg has accuracy on validation 0.8326490713587488
log reg has precision on validation 0.9527789385708316
log reg has recall on validation 0.7542176645716177
              precision    recall  f1-score   support

          -1       0.73      0.95      0.82      2092
           1       0.95      0.75      0.84      3023

   micro avg       0.83      0.83      0.83      5115
   macro avg       0.84      0.85      0.83      5115
weighted avg       0.86      0.83      0.83      5115

[[1979  113]
 [ 743 2280]]
log reg has accuracy on testing 0.7816226783968719
log reg has precision on testing 0.9796888504753674
log reg has recall on testing 0.6793527120167815
              precision    recall  f1-score   support

          -1       0.62      0.97      0.76      1778
           1       0.98      0.68      0.80      3337

   micro avg       0.78      0.78      0.78      5115
   macro avg       0.80      0.83      0.78      5115
weighted avg       0.85      0.78      0.79      5115

[[1731

/local/java/python-pip-packages.cs909/lib64/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


svm has accuracy on validation 0.8502443792766373
svm has precision on validation 0.8490566037735849
svm has recall on validation 0.9080383724776712
              precision    recall  f1-score   support

          -1       0.85      0.77      0.81      2092
           1       0.85      0.91      0.88      3023

   micro avg       0.85      0.85      0.85      5115
   macro avg       0.85      0.84      0.84      5115
weighted avg       0.85      0.85      0.85      5115

[[1604  488]
 [ 278 2745]]
svm has accuracy on testing 0.3478005865102639
svm has precision on testing 1.0
svm has recall on testing 0.00029967036260113877
              precision    recall  f1-score   support

          -1       0.35      1.00      0.52      1778
           1       1.00      0.00      0.00      3337

   micro avg       0.35      0.35      0.35      5115
   macro avg       0.67      0.50      0.26      5115
weighted avg       0.77      0.35      0.18      5115

[[1778    0]
 [3336    1]]
random forest 

In [13]:
print (dt_val_acc, end = "\n")
print (dt_testing_acc, end = " ")

[0.8, 0.824, 0.819, 0.816, 0.817, 0.822, 0.781, 0.832, 0.821]
[0.246, 0.22, 0.605, 0.439, 0.348, 0.508, 0.647, 0.804, 0.826] 

In [14]:
print (log_val_acc, end = "\n")
print (log_testing_acc, end = " ")

[0.818, 0.848, 0.773, 0.852, 0.607, 0.857, 0.843, 0.885, 0.833]
[0.348, 0.348, 0.517, 0.74, 0.587, 0.348, 0.349, 0.349, 0.782] 

In [15]:
print (nb_val_acc, end = "\n")
print (nb_testing_acc, end = " ")

[0.708, 0.489, 0.764, 0.716, 0.549, 0.866, 0.883, 0.677, 0.606]
[0.347, 0.348, 0.395, 0.348, 0.652, 0.348, 0.348, 0.348, 0.348] 

In [16]:
print (svm_val_acc, end = "\n")
print (svm_testing_acc, end = " ")

[0.819, 0.851, 0.848, 0.879, 0.853, 0.855, 0.847, 0.884, 0.85]
[0.348, 0.348, 0.75, 0.349, 0.348, 0.347, 0.348, 0.349, 0.348] 

In [17]:
print (rf_val_acc, end = "\n")
print (rf_testing_acc, end = " ")

[0.852, 0.874, 0.854, 0.882, 0.855, 0.872, 0.866, 0.89, 0.869]
[0.747, 0.755, 0.747, 0.747, 0.748, 0.747, 0.747, 0.747, 0.746] 